In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.14.0


In [2]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(tenant_id="447af084-8821-49a3-890f-1499b3ea7408", # tenantID
                                    service_principal_id="ca5cebc9-2b50-4548-8f97-b99917157e82", # clientId
                                    service_principal_password="ZlfRN3SvgTFOVule.FXTAI__15WPvfSfC.") # clientSecret

In [3]:
from azureml.core import Workspace

ws = Workspace.get(name="mlWorkspace",
                   auth=sp,
                   subscription_id="539bc2da-eefd-4f46-9b84-6499b0af5dd5")
ws.get_details()

{'id': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourceGroups/mlResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/mlWorkspace',
 'name': 'mlWorkspace',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '43f9f1d1-1489-47c1-8c9a-712b4b9e0973',
 'description': '',
 'friendlyName': 'mlWorkspace',
 'creationTime': '2020-09-24T09:07:46.3558506+00:00',
 'containerRegistry': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourceGroups/mlResourceGroup/providers/Microsoft.ContainerRegistry/registries/43f9f1d1148947c18c9a712b4b9e0973',
 'keyVault': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlresourcegroup/providers/microsoft.keyvault/vaults/mlworkspace8762773565',
 'applicationInsights': '/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlresourcegroup/providers/microsoft.insights/components/mlworkspace9232446327',
 'identityPrincipalId': '

In [4]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

mlWorkspace
mlResourceGroup
eastus
539bc2da-eefd-4f46-9b84-6499b0af5dd5


In [5]:
from azureml.train.automl import AutoMLConfig

# task can be one of classification, regression, forecasting
automl_config = AutoMLConfig(task = "classification")

In [6]:
from azureml.core.dataset import Dataset
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = Dataset.Tabular.from_delimited_files(data)

In [7]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automlstep-classification'
project_folder = './automl-pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automlstep-classification,mlWorkspace,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

In [8]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [9]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


Train
This creates a general AutoML settings object. Udacity notes: These inputs must match what was used when training in the portal. label_column_name has to be y for example.

In [17]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

compute_target = ComputeTarget(workspace=ws, name="myamlcompute")

In [18]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target= compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [19]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [20]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [21]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [22]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [c6e9a696][89f75bd0-a6c4-4526-81bd-69e72cf1aed1], (This step will run and generate new outputs)
Submitted PipelineRun 3d130873-c4ae-44ee-8641-9a23f4bc5b02
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-classification/runs/3d130873-c4ae-44ee-8641-9a23f4bc5b02?wsid=/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlResourceGroup/workspaces/mlWorkspace


In [26]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [24]:
pipeline_run.wait_for_completion()

PipelineRunId: 3d130873-c4ae-44ee-8641-9a23f4bc5b02
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-classification/runs/3d130873-c4ae-44ee-8641-9a23f4bc5b02?wsid=/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlResourceGroup/workspaces/mlWorkspace
PipelineRun Status: Running


StepRunId: c12353a9-2b8c-477b-b119-d93b4959cba6
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/automlstep-classification/runs/c12353a9-2b8c-477b-b119-d93b4959cba6?wsid=/subscriptions/539bc2da-eefd-4f46-9b84-6499b0af5dd5/resourcegroups/mlResourceGroup/workspaces/mlWorkspace
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': 'c12353a9-2b8c-477b-b119-d93b4959cba6', 'target': 'myamlcompute', 'status': 'Completed', 'startTimeUtc': '2020-10-07T07:16:24.344737Z', 'endTimeUtc': '2020-10-07T07:25:45.080322Z', 'properties': {'azureml.runsource': 'azureml.StepRun

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [27]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/c12353a9-2b8c-477b-b119-d93b4959cba6/metrics_data, 1 files out of an estimated total of 1


In [28]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,c12353a9-2b8c-477b-b119-d93b4959cba6_10,c12353a9-2b8c-477b-b119-d93b4959cba6_25,c12353a9-2b8c-477b-b119-d93b4959cba6_6,c12353a9-2b8c-477b-b119-d93b4959cba6_0,c12353a9-2b8c-477b-b119-d93b4959cba6_23,c12353a9-2b8c-477b-b119-d93b4959cba6_31,c12353a9-2b8c-477b-b119-d93b4959cba6_37,c12353a9-2b8c-477b-b119-d93b4959cba6_32,c12353a9-2b8c-477b-b119-d93b4959cba6_4,c12353a9-2b8c-477b-b119-d93b4959cba6_12,...,c12353a9-2b8c-477b-b119-d93b4959cba6_16,c12353a9-2b8c-477b-b119-d93b4959cba6_28,c12353a9-2b8c-477b-b119-d93b4959cba6_38,c12353a9-2b8c-477b-b119-d93b4959cba6_18,c12353a9-2b8c-477b-b119-d93b4959cba6_5,c12353a9-2b8c-477b-b119-d93b4959cba6_24,c12353a9-2b8c-477b-b119-d93b4959cba6_17,c12353a9-2b8c-477b-b119-d93b4959cba6_33,c12353a9-2b8c-477b-b119-d93b4959cba6_19,c12353a9-2b8c-477b-b119-d93b4959cba6_26
recall_score_micro,[0.8880121396054628],[0.906525037936267],[0.8977238239757208],[0.9116843702579667],[0.898937784522003],[0.8446130500758725],[0.9119878603945372],[0.9101669195751139],[0.8968133535660091],[0.9004552352048558],...,[0.7056145675265554],[0.8986342943854325],[0.910773899848255],[0.8397572078907436],[0.8995447647951441],[0.9077389984825494],[0.8986342943854325],[0.908649468892261],[0.8880121396054628],[0.8946889226100152]
f1_score_micro,[0.8880121396054628],[0.9065250379362669],[0.8977238239757208],[0.9116843702579667],[0.8989377845220029],[0.8446130500758725],[0.9119878603945372],[0.9101669195751139],[0.8968133535660091],[0.9004552352048558],...,[0.7056145675265554],[0.8986342943854325],[0.9107738998482551],[0.8397572078907436],[0.8995447647951441],[0.9077389984825494],[0.8986342943854325],[0.908649468892261],[0.8880121396054628],[0.8946889226100152]
average_precision_score_weighted,[0.9309982391472723],[0.9422752255464003],[0.9340177057494609],[0.9531771295804466],[0.9216976796034855],[0.9374617574551208],[0.9536766615323959],[0.9469856151812283],[0.9373211977360516],[0.9395303044320702],...,[0.9220170671430082],[0.8420288250935615],[0.9530082656773201],[0.9358395797664488],[0.9259460395255158],[0.9533076386610473],[0.932694148003343],[0.9513421484543777],[0.9246058794409016],[0.9369865192212906]
f1_score_macro,[0.4703423886834914],[0.7004312024313583],[0.6263947343363969],[0.7653697272147331],[0.6100978313360902],[0.7310694605448241],[0.7558510910112933],[0.7318380580337505],[0.7114223480633382],[0.6476560829173383],...,[0.5852603031246756],[0.6412107673609481],[0.7583211587052583],[0.7199806856591018],[0.6170656133421111],[0.7539474330631295],[0.6665084859988897],[0.7469309675333606],[0.4703423886834914],[0.5747772012015496]
precision_score_macro,[0.4440060698027314],[0.7900084518428692],[0.771526544069397],[0.7819118765348991],[0.8022620904836193],[0.6948968457380669],[0.7872435835838584],[0.7915103561961927],[0.7402226496473743],[0.7814639006094702],...,[0.5975396961759405],[0.7698781059540554],[0.7810412995272435],[0.6859633895922105],[0.8017560826299885],[0.7710332748828371],[0.7572605574339628],[0.7767506466136603],[0.4440060698027314],[0.7806809274686013]
log_loss,[0.2605168600296416],[0.21046892934872094],[0.2371632113052707],[0.17775706110025447],[0.2637928922497915],[0.4005407638820437],[0.1804532417662935],[0.3314315185536013],[0.33379726243885416],[0.2337331353599952],...,[0.5259344216697431],[3.1631079115857723],[0.20209310157492416],[0.42369416034276713],[0.24894413046250094],[0.1800272325961882],[0.2355495518342236],[0.18598497314459558],[0.2651845703884316],[0.22108441340331933]
balanced_accuracy,[0.5],[0.6608085253784868],[0.5942781010175104],[0.7513392683482543],[0.5807520464131504],[0.8556702176727851],[0.7325640412931812],[0.6971989285853214],[0.6908642633931465],[0.6112097501699556],...,[0.7229363134369553],[0.6066320642700617],[0.7401694368960094],[0.8351741326709234],[0.5858303371140341],[0.7396447511980246],[0.6314988320764958],[0.724763683043529],[0.5],[0.5570453295100278]
precision_score_weighted,[0.788565560086672],[0.8924260904268563],[0.877014103638037],[0.9072720074188747],[0.8816199555417726],[0

In [29]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/c12353a9-2b8c-477b-b119-d93b4959cba6/model_data, 1 files out of an estimated total of 1


In [30]:
import pickle
import azureml.train.automl

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [31]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('0',
                                             Pipeline(memory=None,
                                                      steps=[('maxabsscaler',
                                                              MaxAbsScaler(copy=True)),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(boosting_type='gbdt',
                                                          

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [32]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [33]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [34]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28512,746
1,1531,2161


In [35]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,8654f019-1a80-40de-a83d-9714c0e76995,Active,REST Endpoint


In [36]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [37]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [38]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  9902b1da-2ed4-4c84-befa-b93da197a69b


In [39]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …